In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('instance').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/21 23:30:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Setup datasets to perform joins

In [11]:
import datetime
from pyspark.sql import Row

courses = [
    {
        'course_id': 1,
        'course_title': 'Mastering Python',
        'course_published_dt': datetime.date(2021, 1, 14),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2021, 2, 18, 16, 57, 25)
    },
    {
        'course_id': 2,
        'course_title': 'Data Engineering Essentials',
        'course_published_dt': datetime.date(2021, 2, 10),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2021, 3, 5, 12, 7, 33)
    },
    {
        'course_id': 3,
        'course_title': 'Mastering Pyspark',
        'course_published_dt': datetime.date(2021, 1, 7),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2021, 4, 6, 10, 5, 42)
    },
    {
        'course_id': 4,
        'course_title': 'AWS Essentials',
        'course_published_dt': datetime.date(2021, 3, 19),
        'is_active': False,
        'last_updated_ts': datetime.datetime(2021, 4, 10, 2, 25, 36)
    },
    {
        'course_id': 5,
        'course_title': 'Docker 101',
        'course_published_dt': datetime.date(2021, 2, 28),
        'is_active': True,
        'last_updated_ts': datetime.datetime(2021, 3, 21, 7, 18, 52)
    }
]

#both work
courses_df = spark.createDataFrame([Row(**course) for course in courses]) # maintained column order
#courses_df = spark.createDataFrame(courses)
courses_df.printSchema()
courses_df.show()

root
 |-- course_id: long (nullable = true)
 |-- course_title: string (nullable = true)
 |-- course_published_dt: date (nullable = true)
 |-- is_active: boolean (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)

+---------+--------------------+-------------------+---------+-------------------+
|course_id|        course_title|course_published_dt|is_active|    last_updated_ts|
+---------+--------------------+-------------------+---------+-------------------+
|        1|    Mastering Python|         2021-01-14|     true|2021-02-18 16:57:25|
|        2|Data Engineering ...|         2021-02-10|     true|2021-03-05 12:07:33|
|        3|   Mastering Pyspark|         2021-01-07|     true|2021-04-06 10:05:42|
|        4|      AWS Essentials|         2021-03-19|    false|2021-04-10 02:25:36|
|        5|          Docker 101|         2021-02-28|     true|2021-03-21 07:18:52|
+---------+--------------------+-------------------+---------+-------------------+



In [3]:
users = [{
  "user_id": 1,
  "user_first_name": "Sandra",
  "user_last_name": "Karpov",
  "user_email": "skarpov0@ovh.net"
}, {
  "user_id": 2,
  "user_first_name": "Kari",
  "user_last_name": "Dearth",
  "user_email": "kdearth1@so-net.ne.jp"
}, {
  "user_id": 3,
  "user_first_name": "Joanna",
  "user_last_name": "Spennock",
  "user_email": "jspennock2@redcross.org"
}, {
  "user_id": 4,
  "user_first_name": "Hirsch",
  "user_last_name": "Conaboy",
  "user_email": "hconaboy3@barnesandnoble.com"
}, {
  "user_id": 5,
  "user_first_name": "Loreen",
  "user_last_name": "Malin",
  "user_email": "lmalin4@independent.co.uk"
}, {
  "user_id": 6,
  "user_first_name": "Augy",
  "user_last_name": "Christon",
  "user_email": "achriston5@mlb.com"
}, {
  "user_id": 7,
  "user_first_name": "Trudey",
  "user_last_name": "Choupin",
  "user_email": "tchoupin6@de.vu"
}, {
  "user_id": 8,
  "user_first_name": "Nadine",
  "user_last_name": "Grimsdell",
  "user_email": "ngrimsdell7@sohu.com"
}, {
  "user_id": 9,
  "user_first_name": "Vassily",
  "user_last_name": "Tamas",
  "user_email": "vtamas8@businessweek.com"
}, {
  "user_id": 10,
  "user_first_name": "Wells",
  "user_last_name": "Simpkins",
  "user_email": "wsimpkins9@amazon.co.uk"
}]

users_df = spark.createDataFrame([Row(**user) for user in users])
users_df.printSchema()
users_df.show()

root
 |-- user_id: long (nullable = true)
 |-- user_first_name: string (nullable = true)
 |-- user_last_name: string (nullable = true)
 |-- user_email: string (nullable = true)

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+-------

In [5]:
course_enrolments = [{
  "course_enrolment_id": 1,
  "user_id": 10,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 2,
  "user_id": 5,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 3,
  "user_id": 7,
  "course_id": 5,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 4,
  "user_id": 9,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 5,
  "user_id": 8,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 6,
  "user_id": 5,
  "course_id": 5,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 7,
  "user_id": 4,
  "course_id": 5,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 8,
  "user_id": 7,
  "course_id": 3,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 9,
  "user_id": 8,
  "course_id": 5,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 10,
  "user_id": 3,
  "course_id": 3,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 11,
  "user_id": 7,
  "course_id": 5,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 12,
  "user_id": 3,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 13,
  "user_id": 5,
  "course_id": 2,
  "price_paid": 9.99
}, {
  "course_enrolment_id": 14,
  "user_id": 4,
  "course_id": 3,
  "price_paid": 10.99
}, {
  "course_enrolment_id": 15,
  "user_id": 8,
  "course_id": 2,
  "price_paid": 9.99
}]

course_enrolments_df = spark.createDataFrame([Row(**ce) for ce in course_enrolments])
course_enrolments_df.printSchema()
course_enrolments_df.show()

root
 |-- course_enrolment_id: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- course_id: long (nullable = true)
 |-- price_paid: double (nullable = true)

+-------------------+-------+---------+----------+
|course_enrolment_id|user_id|course_id|price_paid|
+-------------------+-------+---------+----------+
|                  1|     10|        2|      9.99|
|                  2|      5|        2|      9.99|
|                  3|      7|        5|     10.99|
|                  4|      9|        2|      9.99|
|                  5|      8|        2|      9.99|
|                  6|      5|        5|     10.99|
|                  7|      4|        5|     10.99|
|                  8|      7|        3|     10.99|
|                  9|      8|        5|     10.99|
|                 10|      3|        3|     10.99|
|                 11|      7|        5|     10.99|
|                 12|      3|        2|      9.99|
|                 13|      5|        2|      9.99|
|          

## Overview of join using Spark Data Frames
* INNER join
* LEFT or RIGHT outer join
* FULL outer join
* CROSS join - cartesian product
* Spark Data Frame `.join` and `.crossJoin` function
* semi or anti

## Aliases on Spark Data Frame
* aliases can be defined on top of Spark Data Frame - `alias` function

In [20]:
type(courses_df.alias('c'))

pyspark.sql.dataframe.DataFrame

In [27]:
courses_df.alias('c').select('c.course_id').show()
#courses_df.alias('c').select(c.course_id).show() # does not work
courses_df.alias('c').select('c.*').show()

+---------+
|course_id|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
+---------+

+---------+--------------------+-------------------+---------+-------------------+
|course_id|        course_title|course_published_dt|is_active|    last_updated_ts|
+---------+--------------------+-------------------+---------+-------------------+
|        1|    Mastering Python|         2021-01-14|     true|2021-02-18 16:57:25|
|        2|Data Engineering ...|         2021-02-10|     true|2021-03-05 12:07:33|
|        3|   Mastering Pyspark|         2021-01-07|     true|2021-04-06 10:05:42|
|        4|      AWS Essentials|         2021-03-19|    false|2021-04-10 02:25:36|
|        5|          Docker 101|         2021-02-28|     true|2021-03-21 07:18:52|
+---------+--------------------+-------------------+---------+-------------------+



## Inner join between Spark Data Frames

In [35]:
users_df. \
    join(course_enrolments_df, ['user_id']). \
    show() # 1 user_id column only
# joining on the column with the same name in both data frames
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id). \
    show() # 2 user_id columns

+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|        3|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  2|        2|      9.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  6|        5|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|            

In [36]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id). \
    select(users_df['*'], course_enrolments_df['course_id'], course_enrolments_df['course_enrolment_id']). \ 
    show() # * as a column list for the data frame

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|                  6|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|        5|         

In [41]:
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id). \
    select('u.*', 'ce.course_id', 'ce.course_enrolment_id'). \
    show() 
# works also without specifying alias for the unique column names - ce in this case can be skipped
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show() # user_id is ambiguous, so alias or data frame name is required

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|                  6|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|        5|         

In [46]:
# alias works in groupBy as well
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    groupBy(users_df.user_id). \
    count(). \
    show()
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    groupBy('u.user_id'). \
    count(). \
    show()

+-------+-----+
|user_id|count|
+-------+-----+
|      3|    2|
|      4|    2|
|      5|    3|
|      7|    3|
|      8|    3|
|      9|    1|
|     10|    1|
+-------+-----+

+-------+-----+
|user_id|count|
+-------+-----+
|      3|    2|
|      4|    2|
|      5|    3|
|      7|    3|
|      8|    3|
|      9|    1|
|     10|    1|
+-------+-----+



In [45]:
users_df. \
    join(course_enrolments_df, 'user_id'). \
    groupBy('user_id'). \
    count(). \
    show()

+-------+-----+
|user_id|count|
+-------+-----+
|      3|    2|
|      4|    2|
|      5|    3|
|      7|    3|
|      8|    3|
|      9|    1|
|     10|    1|
+-------+-----+



## Outer join using left between Spark Data Frames

In [48]:
users_df.join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left').show()
users_df.join(course_enrolments_df, 'user_id', 'left_outer').show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|               NULL|   NULL|     NULL|      NULL|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|               NULL|   NULL|     NULL|      NULL|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|      3|        2|      9.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|      3|        3|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|      4|        3|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|      4|        5|     10.99|
|

In [6]:
users_df. \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    select(users_df['*'], course_enrolments_df['course_id'], course_enrolments_df['course_enrolment_id']). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     NULL|               NULL|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|     NULL|               NULL|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|         

In [11]:
users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     NULL|               NULL|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|     NULL|               NULL|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|         

In [16]:
users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    where('course_enrolment_id is null'). \
    show()
users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    where(course_enrolments_df.course_enrolment_id.isNull()). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     NULL|               NULL|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|     NULL|               NULL|
|      6|           Augy|      Christon|  achriston5@mlb.com|     NULL|               NULL|
+-------+---------------+--------------+--------------------+---------+-------------------+

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     NULL|        

In [51]:
from pyspark.sql.functions import count, sum

users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    groupBy('user_id'). \
    agg(count('course_enrolment_id')). \
    show()

from pyspark.sql.functions import when, sum
users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    groupBy('user_id'). \
    agg(sum(when(course_enrolments_df.user_id.isNull(), 0).otherwise(1)).alias('course_count')). \
    show()
users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    groupBy('user_id'). \
    agg(sum(when(course_enrolments_df.course_enrolment_id.isNull(), 0).otherwise(1)).alias('course_count')). \
    show()

+-------+--------------------------+
|user_id|count(course_enrolment_id)|
+-------+--------------------------+
|      1|                         0|
|      2|                         0|
|      3|                         2|
|      4|                         2|
|      5|                         3|
|      6|                         0|
|      7|                         3|
|      8|                         3|
|      9|                         1|
|     10|                         1|
+-------+--------------------------+

+-------+------------+
|user_id|course_count|
+-------+------------+
|      1|           0|
|      2|           0|
|      3|           2|
|      4|           2|
|      5|           3|
|      6|           0|
|      7|           3|
|      8|           3|
|      9|           1|
|     10|           1|
+-------+------------+

+-------+------------+
|user_id|course_count|
+-------+------------+
|      1|           0|
|      2|           0|
|      3|           2|
|      4|           

In [53]:
from pyspark.sql.functions import expr

users_df. \
    alias('u'). \
    join(course_enrolments_df, 'user_id', 'left_outer'). \
    groupBy('user_id'). \
    agg(expr('sum(case when course_enrolment_id is null then 0 else 1 end)').alias('course_count')). \
    show()

+-------+------------+
|user_id|course_count|
+-------+------------+
|      1|           0|
|      2|           0|
|      3|           2|
|      4|           2|
|      5|           3|
|      6|           0|
|      7|           3|
|      8|           3|
|      9|           1|
|     10|           1|
+-------+------------+



## Right outer join between Spark Data Frames

In [54]:
course_enrolments_df.join(users_df, 'user_id', 'right_outer').show()

+-------+-------------------+---------+----------+---------------+--------------+--------------------+
|user_id|course_enrolment_id|course_id|price_paid|user_first_name|user_last_name|          user_email|
+-------+-------------------+---------+----------+---------------+--------------+--------------------+
|      1|               NULL|     NULL|      NULL|         Sandra|        Karpov|    skarpov0@ovh.net|
|      2|               NULL|     NULL|      NULL|           Kari|        Dearth|kdearth1@so-net.n...|
|      3|                 12|        2|      9.99|         Joanna|      Spennock|jspennock2@redcro...|
|      3|                 10|        3|     10.99|         Joanna|      Spennock|jspennock2@redcro...|
|      4|                 14|        3|     10.99|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      4|                  7|        5|     10.99|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|                 13|        2|      9.99|         Loreen|       

## Full outer join - datasets

In [2]:
users1 = [
    {
        "email":"alovett0@nsw.gov.au",
        "first_name":"Aundrea",
        "last_name":"Lovett",
        "gender":"Male",
        "ip_address":"62.72.1.143"
    },
    {
        "email":"bjowling1@spiegel.de",
        "first_name":"Bettine",
        "last_name":"Jowling",
        "gender":"Female",
        "ip_address":"26.250.197.47"
    },
    {
        "email":"rablitt2@technorati.com",
        "first_name":"Reggie",
        "last_name":"Ablitt",
        "gender":"Male",
        "ip_address":"104.181.218.238"
    },
    {
        "email":"tgavahan3@printfriendly.com",
        "first_name":"Ted",
        "last_name":"Gavahan",
        "gender":"Female",
        "ip_address":"216.80.86.100"
    },
    {
        "email":"ccastellan4@bloglovin.com",
        "first_name":"Chantal",
        "last_name":"Castellan",
        "gender":"Female",
        "ip_address":"178.93.82.145"
    },
    {
        "email":"hcurrier5@hexun.com",
        "first_name":"Herrick",
        "last_name":"Currier",
        "gender":"Male",
        "ip_address":"98.120.5.78"
    },
    {
        "email":"zlendrem6@columbia.edu",
        "first_name":"Zorina",
        "last_name":"Lendrem",
        "gender":"Female",
        "ip_address":"219.128.213.53"
    },
    {
        "email":"lbutland7@time.com",
        "first_name":"Lilas",
        "last_name":"Butland",
        "gender":"Female",
        "ip_address":"109.110.131.151"
    },
    {
        "email":"palfonsetti8@ask.com",
        "first_name":"Putnam",
        "last_name":"Alfonsetti",
        "gender":"Female",
        "ip_address":"167.97.48.246"
    },
    {
        "email":"hunitt9@bizjournals.com",
        "first_name":"Holden",
        "last_name":"Unitt",
        "gender":"Female",
        "ip_address":"142.228.161.192"
    },
    {
        "email":"dmcmorrana@reference.com",
        "first_name":"Dorice",
        "last_name":"McMorran",
        "gender":"Female",
        "ip_address":"233.1.28.220"
    },
    {
        "email":"afaulconerb@barnesandnoble.com",
        "first_name":"Andris",
        "last_name":"Faulconer",
        "gender":"Female",
        "ip_address":"109.40.175.103"
    },
    {
        "email":"kupexc@sun.com",
        "first_name":"Krispin",
        "last_name":"Upex",
        "gender":"Male",
        "ip_address":"154.110.22.75"
    },
    {
        "email":"fmancktelowd@youku.com",
        "first_name":"Farand",
        "last_name":"Mancktelow",
        "gender":"Genderqueer",
        "ip_address":"190.20.187.10"
    },
    {
        "email":"kdodgshune@google.com",
        "first_name":"Kellyann",
        "last_name":"Dodgshun",
        "gender":"Female",
        "ip_address":"80.247.105.228"
    }
]

from pyspark.sql import Row

users1_df = spark.createDataFrame([Row(**user) for user in users1])
users1_df.printSchema()
users1_df.show()

root
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)



+--------------------+----------+----------+-----------+---------------+
|               email|first_name| last_name|     gender|     ip_address|
+--------------------+----------+----------+-----------+---------------+
| alovett0@nsw.gov.au|   Aundrea|    Lovett|       Male|    62.72.1.143|
|bjowling1@spiegel.de|   Bettine|   Jowling|     Female|  26.250.197.47|
|rablitt2@technora...|    Reggie|    Ablitt|       Male|104.181.218.238|
|tgavahan3@printfr...|       Ted|   Gavahan|     Female|  216.80.86.100|
|ccastellan4@blogl...|   Chantal| Castellan|     Female|  178.93.82.145|
| hcurrier5@hexun.com|   Herrick|   Currier|       Male|    98.120.5.78|
|zlendrem6@columbi...|    Zorina|   Lendrem|     Female| 219.128.213.53|
|  lbutland7@time.com|     Lilas|   Butland|     Female|109.110.131.151|
|palfonsetti8@ask.com|    Putnam|Alfonsetti|     Female|  167.97.48.246|
|hunitt9@bizjourna...|    Holden|     Unitt|     Female|142.228.161.192|
|dmcmorrana@refere...|    Dorice|  McMorran|     Fe

In [4]:
users2 = [{
        "email":"lbutland7@time.com",
        "first_name":"Lilas",
        "last_name":"Butland",
        "gender":"Female",
        "ip_address":"109.110.131.151"
    },
    {
        "email":"palfonsetti8@ask.com",
        "first_name":"Putnam",
        "last_name":"Alfonsetti",
        "gender":"Female",
        "ip_address":"167.97.48.246"
    },
    {
        "email":"hunitt9@bizjournals.com",
        "first_name":"Holden",
        "last_name":"Unitt",
        "gender":"Female",
        "ip_address":"142.228.161.192"
    },
    {
        "email":"dmcmorrana@reference.com",
        "first_name":"Dorice",
        "last_name":"McMorran",
        "gender":"Female",
        "ip_address":"233.1.28.220"
    },
    {
        "email":"afaulconerb@barnesandnoble.com",
        "first_name":"Andris",
        "last_name":"Faulconer",
        "gender":"Female",
        "ip_address":"109.40.175.103"
    },
    {
        "email":"kupexc@sun.com",
        "first_name":"Krispin",
        "last_name":"Upex",
        "gender":"Male",
        "ip_address":"154.110.22.75"
    },
    {
        "email":"fmancktelowd@youku.com",
        "first_name":"Farand",
        "last_name":"Mancktelow",
        "gender":"Genderqueer",
        "ip_address":"190.20.187.10"
    },
    {
        "email":"kdodgshune@google.com",
        "first_name":"Kellyann",
        "last_name":"Dodgshun",
        "gender":"Female",
        "ip_address":"80.247.105.228"
    },
    {
        "email":"kbaressf@geocities.jp",
        "first_name":"Karly",
        "last_name":"Baress",
        "gender":"Female",
        "ip_address":"145.232.153.145"
    },
    {
        "email":"amillinsg@com.com",
        "first_name":"Adelaide",
        "last_name":"Millins",
        "gender":"Female",
        "ip_address":"75.160.220.182"
    },
    {
        "email":"skemsleyh@quantcast.com",
        "first_name":"Shir",
        "last_name":"Kemsley",
        "gender":"Male",
        "ip_address":"234.195.73.177"
    },
    {
        "email":"kchomiszewskii@simplemachines.org",
        "first_name":"Kristo",
        "last_name":"Chomiszewski",
        "gender":"Female",
        "ip_address":"60.91.73.198"
    },
    {
        "email":"rkelwickj@baidu.com",
        "first_name":"Rosemonde",
        "last_name":"Kelwick",
        "gender":"Genderfluid",
        "ip_address":"42.50.134.65"
    }
]

from pyspark.sql import Row
users2_df = spark.createDataFrame([Row(**user) for user in users2])
users2_df.printSchema()
users2_df.show()

root
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ip_address: string (nullable = true)

+--------------------+----------+------------+-----------+---------------+
|               email|first_name|   last_name|     gender|     ip_address|
+--------------------+----------+------------+-----------+---------------+
|  lbutland7@time.com|     Lilas|     Butland|     Female|109.110.131.151|
|palfonsetti8@ask.com|    Putnam|  Alfonsetti|     Female|  167.97.48.246|
|hunitt9@bizjourna...|    Holden|       Unitt|     Female|142.228.161.192|
|dmcmorrana@refere...|    Dorice|    McMorran|     Female|   233.1.28.220|
|afaulconerb@barne...|    Andris|   Faulconer|     Female| 109.40.175.103|
|      kupexc@sun.com|   Krispin|        Upex|       Male|  154.110.22.75|
|fmancktelowd@youk...|    Farand|  Mancktelow|Genderqueer|  190.20.187.10|
|kdodgshune@google...|  Kellyann|    Dodgshu

In [5]:
users1_df.count(), users2_df.count()

(15, 13)

## Full outer join between the Spark Data Frames

In [71]:
users1_df.join(users2_df, 'email', 'outer').count(), \
users1_df.join(users2_df, 'email', 'full').count(), \
users1_df.join(users2_df, 'email', 'full_outer').count(), \
users1_df.join(users2_df, 'email', 'fullouter').count()

(20, 20, 20, 20)

In [75]:
users1_df.join(users2_df, 'email', 'full').show(truncate=False)
users1_df.join(users2_df, users1_df.email==users2_df.email, 'full').show(truncate=False)

+---------------------------------+----------+----------+-----------+---------------+----------+------------+-----------+---------------+
|email                            |first_name|last_name |gender     |ip_address     |first_name|last_name   |gender     |ip_address     |
+---------------------------------+----------+----------+-----------+---------------+----------+------------+-----------+---------------+
|afaulconerb@barnesandnoble.com   |Andris    |Faulconer |Female     |109.40.175.103 |Andris    |Faulconer   |Female     |109.40.175.103 |
|alovett0@nsw.gov.au              |Aundrea   |Lovett    |Male       |62.72.1.143    |NULL      |NULL        |NULL       |NULL           |
|amillinsg@com.com                |NULL      |NULL      |NULL       |NULL           |Adelaide  |Millins     |Female     |75.160.220.182 |
|bjowling1@spiegel.de             |Bettine   |Jowling   |Female     |26.250.197.47  |NULL      |NULL        |NULL       |NULL           |
|ccastellan4@bloglovin.com        

In [80]:
users1_df. \
    join(users2_df, 'email', 'left'). \
    union(users1_df.join(users2_df, 'email', 'right')). \
    show()

# 8 records are filtered out after distinct which is a desired result
users1_df. \
    join(users2_df, 'email', 'left'). \
    union(users1_df.join(users2_df, 'email', 'right')). \
    count(), \
users1_df. \
    join(users2_df, 'email', 'left'). \
    union(users1_df.join(users2_df, 'email', 'right')). \
    distinct(). \
    count()

+--------------------+----------+----------+-----------+---------------+----------+----------+-----------+---------------+
|               email|first_name| last_name|     gender|     ip_address|first_name| last_name|     gender|     ip_address|
+--------------------+----------+----------+-----------+---------------+----------+----------+-----------+---------------+
| alovett0@nsw.gov.au|   Aundrea|    Lovett|       Male|    62.72.1.143|      NULL|      NULL|       NULL|           NULL|
|bjowling1@spiegel.de|   Bettine|   Jowling|     Female|  26.250.197.47|      NULL|      NULL|       NULL|           NULL|
|rablitt2@technora...|    Reggie|    Ablitt|       Male|104.181.218.238|      NULL|      NULL|       NULL|           NULL|
|tgavahan3@printfr...|       Ted|   Gavahan|     Female|  216.80.86.100|      NULL|      NULL|       NULL|           NULL|
|ccastellan4@blogl...|   Chantal| Castellan|     Female|  178.93.82.145|      NULL|      NULL|       NULL|           NULL|
| hcurrier5@hexu

(28, 20)

In [90]:
from pyspark.sql.functions import coalesce
users1_df. \
    join(users2_df, 'email', 'full'). \
    select(
            'email',
            coalesce(users1_df.first_name, users2_df.first_name).alias('first_name'),
            coalesce(users1_df.last_name, users2_df.last_name).alias('last_name'),
            coalesce(users1_df.gender, users2_df.gender).alias('gender'),
            coalesce(users1_df.ip_address, users2_df.ip_address).alias('ip_address'),
          ). \
    show()
#using aliases
users1_df.alias('u1'). \
    join(users2_df.alias('u2'), 'email', 'full'). \
    select(
            'email',
            coalesce('u1.first_name', 'u2.first_name').alias('first_name'),
            coalesce('u1.last_name', 'u2.last_name').alias('last_name'),
            coalesce('u1.gender', 'u2.gender').alias('gender'),
            coalesce('u1.ip_address', 'u2.ip_address').alias('ip_address'),
          ). \
    show()

+--------------------+----------+------------+-----------+---------------+
|               email|first_name|   last_name|     gender|     ip_address|
+--------------------+----------+------------+-----------+---------------+
|afaulconerb@barne...|    Andris|   Faulconer|     Female| 109.40.175.103|
| alovett0@nsw.gov.au|   Aundrea|      Lovett|       Male|    62.72.1.143|
|   amillinsg@com.com|  Adelaide|     Millins|     Female| 75.160.220.182|
|bjowling1@spiegel.de|   Bettine|     Jowling|     Female|  26.250.197.47|
|ccastellan4@blogl...|   Chantal|   Castellan|     Female|  178.93.82.145|
|dmcmorrana@refere...|    Dorice|    McMorran|     Female|   233.1.28.220|
|fmancktelowd@youk...|    Farand|  Mancktelow|Genderqueer|  190.20.187.10|
| hcurrier5@hexun.com|   Herrick|     Currier|       Male|    98.120.5.78|
|hunitt9@bizjourna...|    Holden|       Unitt|     Female|142.228.161.192|
|kbaressf@geocitie...|     Karly|      Baress|     Female|145.232.153.145|
|kchomiszewskii@si...|   

## Overview of broadcast join in Spark Data Frames
* known as map side or replicated join
* the similar data set will be broadcasted to all exetors in the cluster
* the size of the smaller data set is driven by the `spark.sql.autoBroadcastJoinThreshold`,
* broadcast join can be performed when the smaller data set size is grate than `spark.sql.autoBroadcastJoinThreshold` by using `broadcast` function from `pyspark.sql.functions`
* broadcast join can be disabled by setting `spark.sql.autoBroadcastJoinThreshold` value to 0
* if broadcast join is disabled it results in reduce side join
* make sure to setup multi-node cluster with 28GB memory and 4 cores each. Configure scaling between 2 and 4 nodes. Driver can be of minimum configuration.

In [92]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760b'

In [96]:
#disable broadcast join by setting the value to 0 - reduce side join only
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '0')
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'0'

In [9]:
# resetting back to original value
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '10485760b')
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760b'

In [10]:
users1_df.join(users2_df, 'email').explain()

from pyspark.sql.functions import broadcast
broadcast(users1_df).join(users2_df, 'email').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [email#0, first_name#1, last_name#2, gender#3, ip_address#4, first_name#63, last_name#64, gender#65, ip_address#66]
   +- SortMergeJoin [email#0], [email#62], Inner
      :- Sort [email#0 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(email#0, 200), ENSURE_REQUIREMENTS, [plan_id=336]
      :     +- Filter isnotnull(email#0)
      :        +- Scan ExistingRDD[email#0,first_name#1,last_name#2,gender#3,ip_address#4]
      +- Sort [email#62 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(email#62, 200), ENSURE_REQUIREMENTS, [plan_id=337]
            +- Filter isnotnull(email#62)
               +- Scan ExistingRDD[email#62,first_name#63,last_name#64,gender#65,ip_address#66]


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [email#0, first_name#1, last_name#2, gender#3, ip_address#4, first_name#63, last_name#64, gender#65, ip_address#66]
   +- BroadcastHashJoin [email#0], 

## Cross join using Spark Data Frames

In [24]:
users1_df.count(), \
users2_df.count(), \
users1_df.count() * users2_df.count()

(15, 13, 195)

In [22]:
# crossJoin or inner join without join condition result is cartesian product
users1_df.join(users2_df, 'email').count(), \
users1_df.crossJoin(users2_df).count(), \
users1_df.join(users2_df).count(), \
users1_df.join(users2_df, None, 'cross').count(), \
users1_df.join(users2_df, how='cross').count()

(8, 195, 195, 195, 195)